# Momo Talent 2024

## 1. Data Source

- Mobile Topup (Topup service) is one of MoMo's most important services with a large proportion of users and revenue. Topup service on the MoMo app is displayed under two icons: 'Nạp tiền điện thoại' and 'Mua mã thẻ di động'.														
- You are given the historical daily transactions data during Jan-Dec 2020 (table 'Transactions'), the users' demographic information (table 'User_info'), and the percentage of commission paid by the Telco's merchants to MoMo (table 'Commission').														
- With the given data (see other yellow tabs in this file labeled [Data]), take note of and clean the data as necessary, and accomplish the following requirements.														

### Transactions

| user_id | order_id | Date       | Amount  | Merchant_id | Purchase_status |
|---------|----------|------------|---------|-------------|-----------------|
| 123     | 234      | 2020-01-17 | 100,000 | 12          | Mua hộ          |
| ...     | ...      | ...        | ...     | ...         | ...             |

**Schema:**

- `User_id`: each user in MoMo will be given a unique id
- `Order_id`: each transaction will be given a unique id
- `Date`: date when the transaction took place
- `Amount`: amount of money that user spent on that transaction (VND)
- `Merchant_id`: each merchant will be given a unique id
- `Purchase_status`: if the user uses the account to buy a card for another user, it will be labeled 'Mua hộ'


### User_Info

| User_id | First_tran_date | Location | Age | Gender |
|---------|------------------|----------|-----|--------|
| 123     | 2020-01-17       | HCMC     | >37 | MALE   |
| ...     | ...              | ...      | ... | ...    |

**Schema:**

- `User_id`: each user in MoMo will be given a unique id
- `First_tran_date`: the first day of using Topup service
- `Gender`: Male/Female
- `Age`: user's age has been divided into groups
- `Location`: user's location on the ID card


### Commission

| Merchant_name | Merchant_id | Rate_pct |
|---------------|-------------|----------|
| Viettel       | 12          | 2        |
| Mobifone      | 13          | 3        |
| ...           | ...         | ...      |

**Schema:**

- `Merchant_name`: name of Telco
- `Merchant_id`: each merchant will be given a unique id
- `Rate_pct`: commission percentage (%) that MoMo earns from the merchant for each transaction amount

### User_Info

| User_id | First_tran_date | Location | Age | Gender |
|---------|------------------|----------|-----|--------|
| 123     | 2020-01-17       | HCMC     | >37 | MALE   |
| ...     | ...              | ...      | ... | ...    |

**Schema:**

- `User_id`: each user in MoMo will be given a unique id
- `First_tran_date`: the first day of using Topup service
- `Gender`: Male/Female
- `Age`: user's age has been divided into groups
- `Location`: user's location on the ID card


## 2. Part A: Data Processing


**Requirement**

1. Using data from the 'Commission' table, add a column `Revenue` in the 'Transactions' table that displays MoMo's earned revenue for each order, and then calculate MoMo's total revenue in January 2020.

2. What is MoMo's most profitable month?

3. What day of the week does MoMo make the most money, on average? The least money?

4. Combined with the 'User_Info' table, add columns: `Age`, `Gender`, `Location`, `Type_user` (New/Current) in 'Transactions' table and calculate the total number of new users in December 2020.

    (New = the transaction was in the month of the first time the user used Topup; Current = the user had used Topup before that month)


### 2.1. Loading Data

In [10]:
import pandas as pd

# Load the uploaded CSV files
transactions_df = pd.read_csv('data_transactions1.csv')
user_info_df = pd.read_csv('data_user_info1.csv')
commission_df = pd.read_csv('data_commission1.csv')

# Display the first few rows of each DataFrame to understand their structure and contents
transactions_df.head(), user_info_df.head(), commission_df.head()


(    user_id    order_id                 Date  Amount  Merchant_id  \
 0  21269588  4169517626  2020-01-01 00:00:00   10000           13   
 1  28097592  4170276686  2020-01-01 00:00:00   20000           13   
 2  47435144  4166729310  2020-01-01 00:00:00   10000           12   
 3  29080935  4174460303  2020-01-01 00:00:00   10000           13   
 4  14591075  4168216749  2020-01-01 00:00:00   10000           12   
 
   Purchase_status  
 0         Unknown  
 1         Unknown  
 2         Unknown  
 3         Unknown  
 4         Unknown  ,
     User_id First_tran_date          Location       Age  Gender
 0  41654498      2018-05-02  Ho Chi Minh City  33_to_37    Male
 1  51276281      2019-12-08      Other Cities   unknown    Male
 2  49152375      2019-12-20      Other Cities  23_to_27    Male
 3   5971050      2019-08-01  Ho Chi Minh City  28_to_32    Male
 4  48134464      2019-08-23      Other Cities  18_to_22  Female,
   Merchant_name  Merchant_id  Rate_pct
 0       Viettel    

### 2.2 Handle missing value

In [11]:
# Fixing the date formats
# Convert dates in the format 'DD/MM/YYYY' to 'YYYY-MM-DD'

# Function to convert date format if necessary
def convert_date(date):
    try:
        return pd.to_datetime(date, format='%Y-%m-%d')
    except ValueError:
        return pd.to_datetime(date, format='%d/%m/%Y')

# Apply the conversion function to the Date column
transactions_df['Date'] = transactions_df['Date'].apply(convert_date)

# Verify the Date column is now consistent
transactions_df.head()


,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,10000,13,Unknown
1,28097592,4170276686,2020-01-01,20000,13,Unknown
2,47435144,4166729310,2020-01-01,10000,12,Unknown
3,29080935,4174460303,2020-01-01,10000,13,Unknown
4,14591075,4168216749,2020-01-01,10000,12,Unknown


In [12]:
# Verify consistency of merchant IDs between transactions and commission data
consistent_merchant_ids = transactions_df['Merchant_id'].isin(commission_df['Merchant_id'])
inconsistent_merchants = transactions_df[~consistent_merchant_ids]['Merchant_id'].unique()

# If there are inconsistent merchants, display them
if len(inconsistent_merchants) > 0:
    inconsistent_merchants = pd.DataFrame(inconsistent_merchants, columns=['Inconsistent Merchant_id'])
    import ace_tools as tools; tools.display_dataframe_to_user(name="Inconsistent Merchants", dataframe=inconsistent_merchants)

# If there are no inconsistent merchants, display a message
inconsistent_merchants


array([], dtype=int64)

In [13]:
# Handle the out-of-bounds datetime values by setting them to a default date or handling them as missing
# Setting them to a default date, e.g., '1900-01-01' for clarity

user_info_df['First_tran_date'] = pd.to_datetime(user_info_df['First_tran_date'], errors='coerce')
user_info_df['First_tran_date'].fillna(pd.to_datetime('1900-01-01'), inplace=True)

# Verify the date column is now consistent
user_info_df['First_tran_date'] = pd.to_datetime(user_info_df['First_tran_date'], format='%Y-%m-%d')
user_info_df.head()


,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,Ho Chi Minh City,33_to_37,Male
1,51276281,2019-12-08,Other Cities,unknown,Male
2,49152375,2019-12-20,Other Cities,23_to_27,Male
3,5971050,2019-08-01,Ho Chi Minh City,28_to_32,Male
4,48134464,2019-08-23,Other Cities,18_to_22,Female


Cleaning Gender and Location

In [14]:
# Check distinct values in the 'Location' and 'Gender' columns before cleaning
unique_locations_before = user_info_df['Location'].unique()
unique_genders_before = user_info_df['Gender'].unique()

print("Unique locations before cleaning:", unique_locations_before)
print("Unique genders before cleaning:", unique_genders_before)

# Define the custom mapping for standardizing Location
location_mapping_custom = {
    'HCMC': 'Ho Chi Minh City',
    'Ho Chi Minh City': 'Ho Chi Minh City',
    'Other': 'Other Cities',
    'Other Cities': 'Other Cities',
    'HN': 'Hanoi',
    'Unknown': 'Unknown'
}

# Define the custom mapping for standardizing Gender
gender_mapping_custom = {
    'MALE': 'Male',
    'M': 'Male',
    'Nam': 'Male',
    'male': 'Male',
    'FEMALE': 'Female',
    'F': 'Female',
    'female': 'Female',
    'f': 'Female',
    'Nữ': 'Female'
}

# Apply the mappings to the 'Location' and 'Gender' columns
user_info_df['Location'] = user_info_df['Location'].replace(location_mapping_custom)
user_info_df['Gender'] = user_info_df['Gender'].replace(gender_mapping_custom)

# Check distinct values in the 'Location' and 'Gender' columns after cleaning
unique_locations_after = user_info_df['Location'].unique()
unique_genders_after = user_info_df['Gender'].unique()

print("Unique locations after cleaning:", unique_locations_after)
print("Unique genders after cleaning:", unique_genders_after)

# Save the cleaned data to the original CSV file
user_info_df.to_csv('data_user_info1.csv', index=False)

Unique locations before cleaning: ['Ho Chi Minh City' 'Other Cities' 'Hanoi' 'Unknown']
Unique genders before cleaning: ['Male' 'Female']
Unique locations after cleaning: ['Ho Chi Minh City' 'Other Cities' 'Hanoi' 'Unknown']
Unique genders after cleaning: ['Male' 'Female']


### 2.3 Calculation

In [15]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np


# Ensure 'Amount' is treated as string first and clean it
transactions_df['Amount'] = transactions_df['Amount'].astype(str).str.replace(',', '').astype(float)

# Merge Transactions with Commission to get Rate_pct
transactions_with_commission = pd.merge(transactions_df, commission_df, on='Merchant_id', how='left')

# Calculate Revenue
transactions_with_commission['Revenue'] = transactions_with_commission['Amount'] * transactions_with_commission['Rate_pct'] / 100

# Convert Date column to datetime
transactions_with_commission['Date'] = pd.to_datetime(transactions_with_commission['Date'], dayfirst=True)

# Task 1: Total Revenue in January 2020
january_2020_transactions = transactions_with_commission[
    (transactions_with_commission['Date'].dt.month == 1) & 
    (transactions_with_commission['Date'].dt.year == 2020)
]
total_revenue_january_2020 = january_2020_transactions['Revenue'].sum()

# Task 2: Calculate monthly revenue
transactions_with_commission['YearMonth'] = transactions_with_commission['Date'].dt.to_period('M')
monthly_revenue = transactions_with_commission.groupby('YearMonth')['Revenue'].sum().reset_index()
monthly_revenue = monthly_revenue.rename(columns={'Revenue': 'Monthly_Revenue'})

# Convert YearMonth to string for merging
monthly_revenue['YearMonth'] = monthly_revenue['YearMonth'].astype(str)
transactions_with_commission['YearMonth'] = transactions_with_commission['YearMonth'].astype(str)

# Merge the monthly revenue back into the transactions DataFrame
transactions_with_monthly_revenue = pd.merge(transactions_with_commission, monthly_revenue, on='YearMonth', how='left')

# Task 3: Calculate revenue grouped by day of the week
transactions_with_monthly_revenue['DayOfWeek'] = transactions_with_monthly_revenue['Date'].dt.day_name()
day_of_week_revenue = transactions_with_monthly_revenue.groupby('DayOfWeek')['Revenue'].sum().reset_index()
day_of_week_revenue = day_of_week_revenue.rename(columns={'Revenue': 'DayOfWeek_Revenue'})

# Task 4: Add columns Age, Gender, Location, Type_user and calculate user-specific revenue
# Merge the transactions data with user info
transactions_with_user_info = pd.merge(transactions_with_monthly_revenue, user_info_df, left_on='user_id', right_on='User_id', how='left')

# Add Type_user column
transactions_with_user_info['First_tran_date'] = pd.to_datetime(transactions_with_user_info['First_tran_date'], dayfirst=True, errors='coerce')
transactions_with_user_info['Type_user'] = np.where(
    transactions_with_user_info['Date'].dt.to_period('M') == transactions_with_user_info['First_tran_date'].dt.to_period('M'),
    'New',
    'Current'
)

# Calculate revenue for each user
user_revenue = transactions_with_user_info.groupby(['user_id', 'Type_user']).agg({'Revenue': 'sum'}).reset_index()
user_revenue = user_revenue.rename(columns={'Revenue': 'User_Revenue'})

# Merge user revenue back to transactions
transactions_with_user_info = pd.merge(transactions_with_user_info, user_revenue, on=['user_id', 'Type_user'], how='left')

# Save to Excel
wb = Workbook()
ws = wb.active
ws.title = "Transactions with User Info"

# Add the DataFrame to the worksheet
for r in dataframe_to_rows(transactions_with_user_info, index=False, header=True):
    ws.append(r)
    
# Add the monthly revenue to another sheet
ws3 = wb.create_sheet(title="Monthly Revenue")
for r in dataframe_to_rows(monthly_revenue, index=False, header=True):
    ws3.append(r)

# Add the day of the week revenue to another sheet
ws2 = wb.create_sheet(title="Day of Week Revenue")
for r in dataframe_to_rows(day_of_week_revenue, index=False, header=True):
    ws2.append(r)

# Save the workbook to a file
output_path = "transactions_with_user_info_complete.xlsx"
wb.save(output_path)
print(f"Excel file saved to {output_path}")

Excel file saved to transactions_with_user_info_complete.xlsx


In [16]:
import pandas as pd

# Load the Excel file
file_path = 'transactions_with_user_info_complete.xlsx'
df = pd.read_excel(file_path, sheet_name='Transactions with User Info', parse_dates=['Date'])

# Question 1: Total revenue in January 2020
january_2020_transactions = df[(df['Date'].dt.month == 1) & (df['Date'].dt.year == 2020)]
total_revenue_january_2020 = january_2020_transactions['Revenue'].sum()

print("Total revenue in January 2020:", total_revenue_january_2020)

# Question 2: Most profitable month
df['YearMonth'] = df['Date'].dt.to_period('M')
monthly_revenue = df.groupby('YearMonth')['Revenue'].sum().reset_index()
most_profitable_month = monthly_revenue.loc[monthly_revenue['Revenue'].idxmax()]

print("Most profitable month:", most_profitable_month)

# Question 3: Revenue grouped by day of the week
df['DayOfWeek'] = df['Date'].dt.day_name()
day_of_week_revenue = df.groupby('DayOfWeek')['Revenue'].mean().reset_index()
most_money_day = day_of_week_revenue.loc[day_of_week_revenue['Revenue'].idxmax()]
least_money_day = day_of_week_revenue.loc[day_of_week_revenue['Revenue'].idxmin()]

print("Day with most money:", most_money_day)
print("Day with least money:", least_money_day)

# Question 4: Total number of new users in December 2020
december_2020_transactions = df[(df['Date'].dt.month == 12) & (df['Date'].dt.year == 2020)]
total_new_users_december_2020 = december_2020_transactions[december_2020_transactions['Type_user'] == 'New']['user_id'].nunique()

print("Total number of new users in December 2020:", total_new_users_december_2020)

Total revenue in January 2020: 1410427.02
Most profitable month: YearMonth      2020-12
Revenue      1764400.0
Name: 11, dtype: object
Day with most money: DayOfWeek      Wednesday
Revenue      1488.467083
Name: 6, dtype: object
Day with least money: DayOfWeek         Sunday
Revenue      1311.313671
Name: 3, dtype: object
Total number of new users in December 2020: 72
